In [2]:
import numpy as np
import emoji,csv
import matplotlib.pyplot as plt
%matplotlib inline


In [21]:
def read_csv(filename = 'data/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

X_train,Y_train = read_csv("/home/yash/DeepLearning/data/train_emoji.csv")
X_test,Y_test = read_csv("/home/yash/DeepLearning/data/tesss.csv")
print(X_train.shape)
print(Y_test.shape)

(132,)
(56,)


In [22]:
maxLen = len(max(X_train, key=lambda x: len(x.split())).split())
print(maxLen)

10


In [50]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": "\u263A",
                    "3": "\u2639",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)])

for idx in range(10):
    print(X_train[idx], label_to_emoji(Y_train[idx]))

never talk to me again ☹
I am proud of your achievements ☺
It is the worst day in my life ☹
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit ☹
congratulations on your acceptance ☺
The assignment is too long  ☹
I want to go play ⚾


In [25]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)
print(Y_oh_test)

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0.

In [3]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

w_to_i, i_to_w, word_map = read_glove_vecs('/home/yash/DeepLearning/data/glove.6B.50d.txt')

In [15]:
len(word_map.keys())

400000

In [17]:
def  sentence_to_avg(sentence: str, word_map: dict) :
    any_word = next(iter(word_map.keys()))

    words = sentence.lower().split()

    avg = np.zeros(word_map[any_word].shape)
    count = 0

    for w in words:
        if w in word_map:
            avg += word_map[w]
            count += 1

    if count > 0:
        avg /= count
    
    return avg

sentence = "I am so happy to be here"
print(len(sentence_to_avg(sentence, word_map)))


50


## Now we have all the important parts required to build the model function()

In [27]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def predict(X, Y, W, b, word_to_vec_map):
    """
    Given X (sentences) and Y (emoji indices), predict emojis and compute the accuracy of your model over the given set.
    
    Arguments:
    X -- input data containing sentences, numpy array of shape (m, None)
    Y -- labels, containing index of the label emoji, numpy array of shape (m, 1)
    
    Returns:
    pred -- numpy array of shape (m, 1) with your predictions
    """
    m = X.shape[0]
    pred = np.zeros((m, 1))
    any_word = list(word_to_vec_map.keys())[0]
    # number of classes  
    n_h = word_to_vec_map[any_word].shape[0] 
    
    for j in range(m):                       # Loop over training examples
        
        # Split jth test example (sentence) into list of lower case words
        words = X[j].lower().split()
        
        # Average words' vectors
        avg = np.zeros((n_h,))
        count = 0
        for w in words:
            if w in word_to_vec_map:
                avg += word_to_vec_map[w]
                count += 1
        
        if count > 0:
            avg = avg / count

        # Forward propagation
        Z = np.dot(W, avg) + b
        A = softmax(Z)
        pred[j] = np.argmax(A)
        
    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
    
    return pred


def model(X:str,Y:int,word_map: dict, learning_rate= 0.01, num_iterations = 500):
    any_word = next(iter(word_map.keys()))

    m = Y.shape[0]
    n_y = len(np.unique(Y)) # No of classes
    n_h = word_map[any_word].shape[0] # Dimensions of Glove vectors

    #Initialize the weights using Xavier initialization
    W = np.random.randn(n_y,n_h) / np.sqrt(n_h)
    b = np.zeros(n_y)

    Y_oh = convert_to_one_hot(Y,n_y)

    for t in range(num_iterations):
        cost = 0
        dW = 0
        db = 0

        for i in range(m):
            avg = sentence_to_avg(X[i], word_map)

            # Forward propagate the avg through the softmax layer
            z = np.add(np.dot(W,avg),b)
            a = softmax(z)

            # Compute cost using the i'th training label's one hot representation and "A" (the output of the softmax)
            cost = -np.sum(np.dot(Y_oh[i], np.log(a)))
            ### END CODE HERE ###
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db
            
        if t % 100 == 0:
            print(f"Epoch {t} has a cost of {cost}")
            pred = predict(X,Y,W,b,word_map)

    return pred, W ,b

In [28]:
pred , W, b = model(X_train,Y_train,word_map)
# print(pred)

Epoch 0 has a cost of 1.6954219635708256
Accuracy: 0.36363636363636365
Epoch 100 has a cost of 0.0638715412685588
Accuracy: 0.9242424242424242
Epoch 200 has a cost of 0.03879139080755904
Accuracy: 0.9621212121212122
Epoch 300 has a cost of 0.031394490231089145
Accuracy: 0.9621212121212122
Epoch 400 has a cost of 0.027338111946778487
Accuracy: 0.9772727272727273


In [3]:
# Predicting on train and test
print("Train set:")
pred_train = predict(X_train,Y_train,W,b,word_map)
print(pred_train)

print("Test set:")
pred_test = predict(X_test,Y_test,W,b,word_map)
print(pred_test)

Train set:


NameError: name 'predict' is not defined

In [30]:
X_my_sentences = np.array(["i treasure you", "i love you", "funny lol", "lets play with a ball", "food is ready", "today is not good"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_map)
print(pred)

Accuracy: 0.8333333333333334
[[0.]
 [0.]
 [2.]
 [1.]
 [4.]
 [2.]]


# Emojifier-V2 Using LSTMs in Keras

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense,Dropout,LSTM, Activation, Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform

2024-06-05 15:54:44.673487: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 15:54:45.092542: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 15:54:45.626503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## What we will be building 
1. An Embedding layer which is the embeddings from the word map padded to the max set length 

## Getting started with the Embeddings layer 
1. Write an sentence_to_index function which takes in sentences and returns a list of indices of the words in the sentence.

In [6]:
def sentence_to_indexes(X,word_to_index,maxLen):
    m = X.shape[0]

    X_indices = np.zeros((m,maxLen))

    for i in range(m):
        sentence_words = X[i].lower().split()
        j = 0

        for w in sentence_words:
            if w in word_to_index:
                X_indices[i,j] = word_to_index[w]
                j += 1

    return X_indices

X1 = np.array(['I love football','I like to eat apple'])
print(sentence_to_indexes(X1,w_to_i,5))

[[185457. 226278. 151266.      0.      0.]
 [185457. 222471. 360915. 132701.  57797.]]


In [18]:
def pretrained_embedding_layer(word_map,w_to_i):
    vocab_size = len(w_to_i) + 1
    any_word = next(iter(word_map.keys()))
    emb_dim = word_map[any_word].shape[0]

    emb_matrix = np.zeros((vocab_size,emb_dim))

    for word,idx in w_to_i.items():
        emb_matrix[idx,:] = word_map[word]


    embedding_layer = Embedding(vocab_size,emb_dim,trainable=False)

    embedding_layer.build((None,))

    embedding_layer.set_weights([emb_matrix])

    return embedding_layer

embedding_layer =  pretrained_embedding_layer(word_map,w_to_i)
print(embedding_layer.get_weights()[0][1][1])
print(embedding_layer.input_dim)
print(embedding_layer.output_dim)


0.39031
400001
50


In [23]:
def Emojify_v2(input_shape,word_map,w_to_i):
    sentence_indices = Input(shape=input_shape,dtype='int32')

    embedding_layer = pretrained_embedding_layer(word_map,w_to_i)

    embeddings = embedding_layer(sentence_indices)

    X = LSTM(units=128,return_sequences=True)(embeddings)

    X = Dropout(rate=0.5)(X)

    X = LSTM(units=128,return_sequences=False)(X)

    X = Dropout(rate=0.5)(X)

    X = Dense(5)(X)

    X = Activation('sigmoid')(X)
    model = Model(inputs= sentence_indices, outputs= X )

    return model


model = Emojify_v2((maxLen,),word_map,w_to_i)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, 10, 50)         │    20,000,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 10, 128)        │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │           645 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,223,927 (77.15 MB)

 Trainable params: 223,877 (874.52 KB)

 Non-trainable params: 20,000,050 (76.29 MB)

In [24]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [27]:
X_train_indices = sentence_to_indexes(X_train,w_to_i,maxLen)
model.fit(X_train_indices,Y_oh_train,epochs=50, batch_size=32, shuffle=True)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0056 
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0046 
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0047
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0035 
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0055 
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0040 
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0034
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0032 
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 1.0000 - loss: 0.0037
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0054 
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.0026
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0070
E

In [29]:
X_test_indices = sentence_to_indexes(X_test, w_to_i,maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8527 - loss: 0.7168  

Test accuracy =  0.8571428656578064


In [35]:
# This code allows you to see the mislabelled examples
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentence_to_indexes(X_test, w_to_i, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Expected emoji::smile: prediction: she got me a nice present	❤️
Expected emoji::disappointed: prediction: work is hard	:smile:
Expected emoji::disappointed: prediction: This girl is messing with me	❤️
Expected emoji::disappointed: prediction: work is horrible	:smile:
Expected emoji::disappointed: prediction: she is a bully	❤️
Expected emoji::disappointed: prediction: My life is so boring	❤️
Expected emoji:⚾ prediction: what is your favorite baseball game	:smile:
Expected emoji::disappointed: prediction: go away	⚾


In [51]:
x_test = np.array(['not feeling happy'])
x_test_indices = sentence_to_indexes(x_test,w_to_i,maxLen)
print(label_to_emoji(np.argmax(model.predict(x_test_indices))))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
☹
